# NYC Facades Condition Alart

workflow:
1. load csv
2. separate 'Safe' and 'Unsafe' building
3. for each group, scrape a photo of the facade according to the building's address
4. split training and testing data
5. build model to predict a if a facade is safe or not

Import libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests as r

## Extract

In [3]:
url_facade_data = "https://data.cityofnewyork.us/resource/xubg-57si.csv"

In [4]:
facade_con = pd.read_csv(url_facade_data)

## Examin

In [5]:
# lift column disply limit to see more columns
pd.set_option('display.max_columns', None)

In [6]:
#preview data
facade_con.head()

,tr6_no,control_no,filing_type,cycle,bin,house_no,street_name,borough,block,lot,sequence_no,submitted_on,current_status,qewi_name,qewi_bus_name,qewi_bus_street_name,qewi_city,qewi_state,qewi_zip,qewi_nys_lic_no,owner_name,owner_bus_name,owner_bus_street_name,owner_city,owner_zip,owner_state,filing_date,filing_status,prior_cycle_filing_date,prior_status,field_inspection_completed_date,qewi_signed_date,late_filing_amt,failure_to_file_amt,failure_to_collect_amt,comments
0,TR6-716580-7A-N1,716580,Auto-Generated,7,1063335,900,RIVERSIDE DRIVE,MANHATTAN,2136,167,NaN,NaN,No Report Filed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Report Filed,NaN,NaN,NaN,NaN,55000.0,19000.0,0.0,NaN
1,TR6-716657-7C-N1,716657,Auto-Generated,7,1087050,116,EAST 125 STREET,MANHATTAN,1773,7,NaN,NaN,UNSAFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Report Filed,NaN,NaN,NaN,NaN,18000.0,5000.0,21000.0,Civil Penalty Stopped on: 3/25/2019
2,TR6-815020-8C-I1,815020,Initial,8,4610766,2,BAY CLUB DRIVE,QUEENS,5893,7501,2.0,2019-07-30 00:00:00,SWARMP,CHARLES MERRITT,"MERRITT ENGINEERING CONSULTING, P.C",28-08 BAYSIDE LANE,FLUSHING,NY,11358,PE - 048838,MICHAEL BRENNAN,THE BAY CLUB CONDON,1 BAY CLUB DR,BAYSIDE,11360,NY,2019-07-30T00:00:00.000,UNSAFE,NaN,NaN,2019-07-11T00:00:00.000,2019-07-24T20:00:00.000,1500.0,0.0,0.0,DUE TO GENERATING ERROR PREMISE RE-ADDED TO FI...
3,TR6-815020-8C-A1,815020,Amended,8,4610766,2,BAY CLUB DRIVE,QUEENS,5893,7501,2.0,2020-01-23 00:00:00,SWARMP,CHARLES MERRITT,"MERRITT ENGINEERING CONSULTING, P.C",28-08 BAYSIDE LANE,FLUSHING,NY,11358,PE - 048838,MICHAEL BRENNAN,THE BAY CLUB CONDON,1 BAY CLUB DR,BAYSIDE,11360,NY,2020-01-23T00:00:00.000,SWARMP,2019-07-30T00:00:00.000,UNSAFE,2019-12-06T00:00:00.000,2020-01-29T19:00:00.000,1500.0,0.0,0.0,DUE TO GENERATING ERROR PREMISE RE-ADDED TO FI...
4,TR6-815020-8C-N1,815020,Auto-Generated,8,4610766,2,BAY CLUB DRIVE,QUEENS,5893,7501,2.0,NaN,SWARMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Report Filed,NaN,NaN,NaN,NaN,1500.0,0.0,0.0,DUE TO GENERATING ERROR PREMISE RE-ADDED TO FI...


In [7]:
# DataFrame data types
facade_con.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   tr6_no                           1000 non-null   object 
 1   control_no                       1000 non-null   int64  
 2   filing_type                      1000 non-null   object 
 3   cycle                            1000 non-null   int64  
 4   bin                              1000 non-null   int64  
 5   house_no                         1000 non-null   object 
 6   street_name                      1000 non-null   object 
 7   borough                          1000 non-null   object 
 8   block                            1000 non-null   int64  
 9   lot                              1000 non-null   int64  
 10  sequence_no                      959 non-null    float64
 11  submitted_on                     957 non-null    object 
 12  current_status       

In [8]:
facade_con.describe(include='all')

,tr6_no,control_no,filing_type,cycle,bin,house_no,street_name,borough,block,lot,sequence_no,submitted_on,current_status,qewi_name,qewi_bus_name,qewi_bus_street_name,qewi_city,qewi_state,qewi_zip,qewi_nys_lic_no,owner_name,owner_bus_name,owner_bus_street_name,owner_city,owner_zip,owner_state,filing_date,filing_status,prior_cycle_filing_date,prior_status,field_inspection_completed_date,qewi_signed_date,late_filing_amt,failure_to_file_amt,failure_to_collect_amt,comments
count,1000,1000.000000,1000,1000.000000,1.000000e+03,1000,1000,1000,1000.000000,1000.000000,959.000000,957,754,958,958,958,958,958,854,854,854,962,962,962,854,854,957,1000,824,823,947,851,994.000000,994.000000,994.000000,181
unique,993,NaN,4,NaN,NaN,606,498,5,NaN,NaN,NaN,281,4,241,254,232,76,4,99,192,550,762,582,78,163,8,277,4,542,3,302,328,NaN,NaN,NaN,154
top,TR6-814993-8A-N1,NaN,Initial,NaN,NaN,20,BROADWAY,MANHATTAN,NaN,NaN,NaN,2020-10-30 00:00:00,UNSAFE,DAVID SALAMON,SALAMON ENGINEERING PLLC,159 WEST 25TH STREET,NEW YORK,NY,10001,PE - 060930,RICHARD MORRISON,PR,24-02 49TH AVENUE,NEW YORK,10016,NY,2020-10-30T00:00:00.000,SWARMP,2013-02-20T00:00:00.000,SAFE,2020-10-28T00:00:00.000,2020-10-27T20:00:00.000,NaN,NaN,NaN,INSPECTION 7/15/15 SUBJECT LETTER SENT 11/3/16
freq,4,NaN,911,NaN,NaN,10,39,643,NaN,NaN,NaN,105,267,33,32,51,497,849,136,32,37,58,24,483,50,829,99,338,22,385,25,22,NaN,NaN,NaN,4
mean,NaN,868428.618000,NaN,8.610000,1.821727e+06,NaN,NaN,NaN,2198.091000,1394.748000,1.201251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12971.579477,4443.661972,2570.422535,NaN
std,NaN,97487.571086,NaN,0.953315,1.189661e+06,NaN,NaN,NaN,2591.546997,2873.461912,0.968264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12137.189095,4072.522949,8831.850400,NaN
min,NaN,600001.000000,NaN,6.000000,1.000005e+06,NaN,NaN,NaN,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
25%,NaN,901855.250000,NaN,9.000000,1.024836e+06,NaN,NaN,NaN,708.500000,15.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2250.000000,0.000000,0.000000,NaN
50%,NaN,906018.500000,NaN,9.000000,1.064327e+06,NaN,NaN,NaN,1341.500000,38.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12000.000000,5000.000000,0.000000,NaN
75%,NaN,911703.500000,NaN,9.000000,2.115555e+06,NaN,NaN,NaN,2463.500000,100.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17250.000000,6000.000000,0.000000,NaN


Are there houses with no adress? - No.

In [9]:
facade_con['house_no'].isna().sum()

0

The possible values 'status' of a building's address

In [10]:
#set(facade_con['house_no'].values.tolist())

We can see that, while there are no missing values, some buildings location are actually a range ("1-10") - probably meaning they are a long *'block'* building. 
This will require attantion when we will try to retrive images from google

### separte 'Safe' and 'Unsafe' building

In [11]:
safe_df = facade_con.loc[(facade_con['filing_status'] == 'SAFE')]
unsafe_df = facade_con.loc[(facade_con['filing_status'] == 'UNSAFE')]

In [12]:
# add  'SWARMP' buildings Safe with a Repair and Maintenance Program (building code classification; New York, NY)
swarmp_df = facade_con.loc[(facade_con['filing_status'] == 'SWARMP')]

In [13]:
# merge SAFE and SWARMP
frames = [safe_df, swarmp_df]
safe_mrg_df = pd.concat(frames, sort=False)

In [14]:
unsafe_df.shape

(292, 36)

In [15]:
safe_df.shape

(326, 36)

In [16]:
swarmp_df.shape

(338, 36)

In [17]:
safe_mrg_df.shape

(664, 36)

*QEWI* means Qualified Exterior Wall Inspector

In [18]:
safe_mrg_df

,tr6_no,control_no,filing_type,cycle,bin,house_no,street_name,borough,block,lot,sequence_no,submitted_on,current_status,qewi_name,qewi_bus_name,qewi_bus_street_name,qewi_city,qewi_state,qewi_zip,qewi_nys_lic_no,owner_name,owner_bus_name,owner_bus_street_name,owner_city,owner_zip,owner_state,filing_date,filing_status,prior_cycle_filing_date,prior_status,field_inspection_completed_date,qewi_signed_date,late_filing_amt,failure_to_file_amt,failure_to_collect_amt,comments
14,TR6-815027-8A-I1,815027,Initial,8,1084518,2937,8 AVENUE,MANHATTAN,2106,3,6.0,2019-11-26 00:00:00,SAFE,KENNETH SANTELLI,KENNETH A SANTELLI,480 NORTH BROADWAY,YONKERS,NY,10701,RA - 028770,LLOYD VALDEZ,NYCHA,90 CHURCH STREET,NEW YORK,10007,NY,2019-11-26T00:00:00.000,SAFE,NaN,NaN,2019-10-16T00:00:00.000,NaN,0.0,0.0,0.0,NaN
21,TR6-816622-8C-S1,816622,Subsequent,8,1085588,550,1st Ave,MANHATTAN,962,7501,5.0,2020-01-15 00:00:00,SAFE,HOWARD ZIMMERMAN,HLZIMMERMAN ARCHITECTS,11 W 30 ST,NY,NY,10001,RA - 014327,RICHARD COHEN,NYU LANGONE MEDICAL CENTER/NYU HOSPITALS CENTER,545 1ST AVENUE,NEW YORK,10016,NY,2020-01-15T00:00:00.000,SAFE,2018-07-02T00:00:00.000,SWARMP,2019-12-13T00:00:00.000,2019-12-31T19:00:00.000,0.0,0.0,0.0,NaN
25,TR6-814977-8C-I2,814977,Initial,8,2013306,333,EAST 181 STREET,BRONX,3143,36,2.0,2017-10-02 00:00:00,SAFE,GEORGE XERAKIAS,GEORGE J. XERAKIAS,160-14 12TH AVENUE,WHITESTONE,NY,11357,PE - 088368,PETER LEPORE,"APARTMENT MANAGEMENT ASSOCIATES, LL",2611 WEST 2ND STREET,BROOKLYN,11223,NY,2017-10-02T00:00:00.000,SAFE,2007-03-21T00:00:00.000,SWARMP,2017-10-02T00:00:00.000,2017-10-03T00:00:00.000,0.0,0.0,0.0,NaN
27,TR6-905332-9A-I1,905332,Initial,9,1045585,1293,YORK AVENUE,MANHATTAN,1464,21,1.0,2020-04-28 00:00:00,SAFE,BRIAN SULLIVAN,SULLIVAN ENGINEERING,409 MINNISINK ROAD,TOTOWA,NJ,07512,PE - 088575,DMITRI KONON,WEILL CORNELL MEDICAL COLLEGE,445 EAST 69TH STREET,NEW YORK,10021,NY,2020-04-28T00:00:00.000,SAFE,2017-05-03T00:00:00.000,SWARMP,2020-03-13T00:00:00.000,2020-05-07T20:00:00.000,0.0,0.0,0.0,NaN
34,TR6-905428-9C-A1,905428,Amended,9,1046650,995,FIFTH AVENUE,MANHATTAN,1492,69,1.0,2020-07-20 00:00:00,SAFE,STEVE LANE,DEVON ARCHITECTS,20 JERUSALEM AVENUE,HICKSVILLE,NY,11801,RA - 020904,EAMON EARLY,995 5th Avenue. LLC,770 LEXINGTON AVENUE,NEW YORK,10065,NY,2020-07-20T00:00:00.000,SAFE,2020-04-30T00:00:00.000,UNSAFE,2020-07-09T00:00:00.000,2020-08-05T20:00:00.000,17750.0,5000.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,TR6-600016-NA-I1,600016,Initial,6,1085816,26,BEAVER STREET,MANHATTAN,11,14,1.0,2006-05-22 00:00:00,SWARMP,THOMAS FENNIMAN,THOMAS A FENNIMAN,1 UNION SQUARE WEST,NEW YORK,NY,NaN,NaN,NaN,"BEAVER TOWERS, INC.",404 PARK AVENUE SOUTH,NY,NaN,NaN,2006-05-22T00:00:00.000,SWARMP,2000-02-29T00:00:00.000,SAFE,2006-04-13T00:00:00.000,2006-05-12T00:00:00.000,3250.0,0.0,6000.0,NaN
991,TR6-600017-NA-I1,600017,Initial,6,1000038,80,BROAD STREET,MANHATTAN,11,21,1.0,2007-08-31 00:00:00,SWARMP,JOHN J HOFFMANN,"HOFFMANN ARCHITECTS,INC",500 FIFTH AVENUE,NEW YORK,NY,NaN,NaN,NaN,"80 BROAD,LLC",770 LEXINGTON AVENUE,NEW YORK,NaN,NaN,2007-08-31T00:00:00.000,SWARMP,2000-02-05T00:00:00.000,SWARMP,2006-06-06T00:00:00.000,2007-07-18T00:00:00.000,1300.0,0.0,0.0,NaN
993,TR6-600019-NA-I1,600019,Initial,6,1000044,5,BROADWAY,MANHATTAN,13,5,1.0,2010-02-12 00:00:00,SWARMP,FRED P ROSLYN,ROSLYN ENGINEERING ASSOCIATES PC,111 JOHN STREET,NEW YORK,NY,NaN,NaN,NaN,BOWLING GREEN.ASSOCIATES PC,160 BROADWAY,NY,NaN,NaN,2010-02-12T00:00:00.000,SWARMP,2000-02-23T00:00:00.000,SWARMP,2009-07-06T00:00:00.000,2009-11-10T00:00:00.000,7750.0,0.0,7000.0,INVOICE #61105521 (SNE)
996,TR6-600094-NA-I1,600094,Initial,6,1000833,76,BEAVER STREET,MANHATTAN,28,12,1.0,2007-02-21 00:00:00,SWARMP,MICHEAL V NATARO,ZASKORSKI NOTARO ARCHITECTS LLP,575 8TH AVENUE SUITE 2212,NY,NY,NaN,NaN,NaN,11 HANOVER SQUARE CORP,855 LEXINGTON AVENUE,NY,NaN,NaN,2007-02-21T00:00:00.000,SWARMP,2000-03-01T00:00:00.000,SAF

## Get the adresses:

For each DF, add an *address* column who's value's a string concatunating the *house_no*, *street_name*, and 'borough' + Ending with "New York, New York"

In [19]:
safe_mrg_df['address'] = safe_mrg_df['house_no'].astype(str) + ' ' + safe_mrg_df['street_name'].astype(str) + ', ' + safe_mrg_df['borough'].astype(str) + ', New York, New York'
unsafe_df['address'] = unsafe_df['house_no'].astype(str) + ' ' + unsafe_df['street_name'].astype(str) + ', ' + unsafe_df['borough'].astype(str) + ', New York, New York'

<ipython-input-19-68e37cf459b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unsafe_df['address'] = unsafe_df['house_no'].astype(str) + ' ' + unsafe_df['street_name'].astype(str) + ', ' + unsafe_df['borough'].astype(str) + ', New York, New York'


## Convert physical addresses to Geographic locations → Latitude and Longitude 

Import libraries

In [20]:
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [21]:
from geopy.extra.rate_limiter import RateLimiter
geocoder = Nominatim(user_agent = 'ironhack') # notice changing the user_agent name in other projects
geocode = RateLimiter(geocoder.geocode, min_delay_seconds=1) # to delay between geocoding calls

In [22]:
# create location column
safe_mrg_df['location'] = safe_mrg_df['address'].apply(geocode)
unsafe_df['location'] = unsafe_df['address'].apply(geocode)
# create longitude,laatitude and altitude from location column (returns tuple)
unsafe_df['point'] = unsafe_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
safe_mrg_df['point'] = safe_mrg_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

<ipython-input-22-9c0d0a73564f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unsafe_df['location'] = unsafe_df['address'].apply(geocode)
<ipython-input-22-9c0d0a73564f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unsafe_df['point'] = unsafe_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)


split point column into latitude, longitude and altitude columns

In [23]:
safe_mrg_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(safe_mrg_df['point'].tolist(), index=safe_mrg_df.index)
unsafe_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(unsafe_df['point'].tolist(), index=unsafe_df.index)

C:\Users\meita\anaconda3\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [25]:
unsafe_df

,tr6_no,control_no,filing_type,cycle,bin,house_no,street_name,borough,block,lot,sequence_no,submitted_on,current_status,qewi_name,qewi_bus_name,qewi_bus_street_name,qewi_city,qewi_state,qewi_zip,qewi_nys_lic_no,owner_name,owner_bus_name,owner_bus_street_name,owner_city,owner_zip,owner_state,filing_date,filing_status,prior_cycle_filing_date,prior_status,field_inspection_completed_date,qewi_signed_date,late_filing_amt,failure_to_file_amt,failure_to_collect_amt,comments,address,location,point,latitude,longitude,altitude
2,TR6-815020-8C-I1,815020,Initial,8,4610766,2,BAY CLUB DRIVE,QUEENS,5893,7501,2.0,2019-07-30 00:00:00,SWARMP,CHARLES MERRITT,"MERRITT ENGINEERING CONSULTING, P.C",28-08 BAYSIDE LANE,FLUSHING,NY,11358,PE - 048838,MICHAEL BRENNAN,THE BAY CLUB CONDON,1 BAY CLUB DR,BAYSIDE,11360,NY,2019-07-30T00:00:00.000,UNSAFE,NaN,NaN,2019-07-11T00:00:00.000,2019-07-24T20:00:00.000,1500.0,0.0,0.0,DUE TO GENERATING ERROR PREMISE RE-ADDED TO FI...,"2 BAY CLUB DRIVE, QUEENS, New York, New York","(2, 24th Avenue, Queens, Queens County, New Yo...","(40.778490765306124, -73.7810555, 0.0)",40.778491,-73.781055,0.0
12,TR6-815026-8B-I1,815026,Initial,8,1015915,60,WEST 36 STREET,MANHATTAN,837,78,1.0,2020-02-18 00:00:00,UNSAFE,DAVID MAY,SUPERSTRUCTURES ENGINEERING + ARCHI,32 AVE OF AMERICAS,NEW YORK,NY,10013,RA - 014625,JADIE SHUM,Hospitality Finance Company One LLC,60 WEST 36TH STREET,NEW YORK,10018,NY,2020-02-18T00:00:00.000,UNSAFE,NaN,NaN,2020-01-21T00:00:00.000,2020-02-16T19:00:00.000,6000.0,1000.0,8000.0,NaN,"60 WEST 36 STREET, MANHATTAN, New York, New York","(36, Sunken Meadow Loop, Manhattan Community B...","(40.79513125, -73.9159314824746, 0.0)",40.795131,-73.915931,0.0
16,TR6-915008-9B-I1,915008,Initial,9,2015210,212,WEST KINGSBRIDGE ROAD,BRONX,3240,37,1.0,2020-05-12 00:00:00,UNSAFE,TARIQ BASHIR,"TARIQ M. BASHIR, P.E.",592 LUCY AVENUE,TEANECK,NJ,07666,PE - 075189,BESNIK CELAJ,"212 KINGSBRIDGE, LLC",754,BRONX,10467,NY,2020-05-12T00:00:00.000,UNSAFE,NaN,NaN,2020-04-18T00:00:00.000,2020-03-05T19:00:00.000,17000.0,6000.0,5000.0,INSPECT 6/3/08 ADD TO UNIVERSE 6/12/18 (NG)AS ...,"212 WEST KINGSBRIDGE ROAD, BRONX, New York, Ne...","(Iglesia de Dios, 212, West Kingsbridge Road, ...","(40.8719965, -73.9047067, 0.0)",40.871997,-73.904707,0.0
17,TR6-915008-9B-S1,915008,Subsequent,9,2015210,212,WEST KINGSBRIDGE ROAD,BRONX,3240,37,1.0,2020-10-12 00:00:00,UNSAFE,HAMIDREZA SAGHARI,NY RESTORATION TECHNOLOGY CORP,1867 AMSTERDAM AVE,NEW YORK,NY,10031,RA - 034963,BESNIK CELAJ,"212 KINGSBRIDGE, LLC",754,BRONX,10467,NY,2020-10-12T00:00:00.000,UNSAFE,2020-05-12T00:00:00.000,UNSAFE,2020-09-16T00:00:00.000,2020-10-09T20:00:00.000,17000.0,6000.0,5000.0,INSPECT 6/3/08 ADD TO UNIVERSE 6/12/18 (NG)AS ...,"212 WEST KINGSBRIDGE ROAD, BRONX, New York, Ne...","(Iglesia de Dios, 212, West Kingsbridge Road, ...","(40.8719965, -73.9047067, 0.0)",40.871997,-73.904707,0.0
29,TR6-905359-9C-I1,905359,Initial,9,1045990,445,EAST 77 STREET,MANHATTAN,1472,22,1.0,2020-09-30 00:00:00,UNSAFE,RUSLAN SOLOVYEV,STRUCTURAL ENGINEERING BURO,304 MAJOR AVENUE,STATEN ISLAND,NY,10305,PE - 093787,MARTIN SCHARF,77 Realty LLC,P.O. BOX 20449,FLORAL PARK,11002,NY,2020-09-30T00:00:00.000,UNSAFE,2014-06-02T00:00:00.000,SWARMP,2020-09-15T00:00:00.000,2020-09-16T20:00:00.000,15000.0,6000.0,0.0,NaN,"445 EAST 77 STREET, MANHATTAN, New York, New York","(77th Street, East 77th Street, Lenox Hill, Ma...","(40.7736399, -73.9598468, 0.0)",40.773640,-73.959847,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,TR6-916947-9B-I1,916947,Initial,9,4206536,135-24,HOOVER AVENUE,QUEENS,9667,15,NaN,2020-05-22 00:00:00,UNSAFE,JOHN MONROE,RAND ENGINEERING & ARCHITECTURE DPC,159 WEST 25TH STREET,NEW YORK,NY,10001,RA - 034900,MARK MOSKOWITZ,Canterbury Realty Associates,50 W 17TH ST,NY,10011,NY,2020-05-22T00:00:00.000,UNSAFE,NaN,NaN,2020-05-13T00:00:00.000,2020-05-14T20:00:00.000,10000.0,6000

remove rows that didn't return a location succusfuly

In [26]:
safe_mrg_df_notnull = safe_mrg_df[pd.notnull(safe_mrg_df["latitude"])]
unsafe_df_notnull = unsafe_df[pd.notnull(unsafe_df["latitude"])]

## Visualize dots

In [27]:
map_save_unsafe = folium.Map(
    location=[40.730610,-73.935242], # (lan, lat) of NYC
    tiles='Stamen Toner',
    zoom_start=12,
)

In [28]:
safe_mrg_df_notnull.apply(lambda row:folium.Circle(location=[row["latitude"], row["longitude"]], color='CornflowerBlue', radius=1.5, popup = row["address"]).add_to(map_save_unsafe), axis=1)
unsafe_df_notnull.apply(lambda row:folium.Circle(location=[row["latitude"], row["longitude"]], color='crimson', radius=1.5, popup = row["address"]).add_to(map_save_unsafe), axis=1)

map_save_unsafe # dispky

In [ ]:
unsafe_df

# Scraping google images using Selenium

In [31]:
unsafe_df['location']

2      (2, 24th Avenue, Queens, Queens County, New Yo...
12     (36, Sunken Meadow Loop, Manhattan Community B...
16     (Iglesia de Dios, 212, West Kingsbridge Road, ...
17     (Iglesia de Dios, 212, West Kingsbridge Road, ...
28     (77th Street, East 77th Street, Lenox Hill, Ma...
                             ...                        
975    (25, Broad Street, Financial District, Manhatt...
977    (38, Exchange Place, Financial District, Manha...
979    (43, Exchange Place, Financial District, Manha...
983    (&pizza, 63, Wall Street, Financial District, ...
990    (Fino Ristorante Italiano, 82, Beaver Street, ...
Name: location, Length: 292, dtype: object

### Import libraries

General

In [32]:
# for printing current time when crashes and waits
import time
import datetime

# for scraping htmls
from bs4 import BeautifulSoup

# to avoid getting blocked
import random

import requests  # for soup

import re # for regex search
import lxml # better then html detection 
import cchardet # to assit in charecter detection

Webdriver specific

In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # for typing
from selenium.webdriver.chrome.options import Options # for loading the page : start only after loading
# for waiting on elemnts to load:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# being able to 'hover' over an elemnt
from selenium.webdriver.common.action_chains import ActionChains

#Chrome.options.add_argument("--headless")

# for opening the web browser
PATH = "C:\\Program Files (x86)\\chromedriver.exe" 

In [34]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("detach", True)
#chrome_options.add_argument("--window-size=1024x1024") # makes window sqaure - not needed


## Functions

In [35]:
gmaps_url_base = 'https://www.google.com/maps/place/'

Initialize the webdriver and accept cookies

In [36]:
def open_gmaps(gmaps_url_base):
    driver = webdriver.Chrome(PATH)
    driver.implicitly_wait(10)
    driver.get(gmaps_url_base)

Click on the 'accept cookies' button at start

In [37]:
def accept_cookies():
    # RUN ONLY ONCE
    # allow time for page to load
    #time.sleep(30)
    #WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]')))
    # we can't find the elemnt without switching to its frame first:
    driver.switch_to.frame(driver.find_element_by_xpath("//iframe[contains(@src, 'consent.google.com')]")) 
    time.sleep(random.uniform(2, 8)) # look human
    driver.find_element_by_xpath('//*[@id="introAgreeButton"]/span/span').click()
    #driver.switch_to().parentFrame(); # go back to defualt pane
    print("CLICKED: cockies")

Use search bar

In [38]:
def search_location(address):
    time.sleep(random.uniform(2, 5)) # look human"
    driver.implicitly_wait(10)
    search_bar = driver.find_element_by_xpath('//*[@id="searchbox-searchbutton"]')
    search_bar.send_keys(f"{address}",Keys.ENTER) # type the 'address' (function input) and press 'Enter'

Try and move to street view

In [39]:
def clk_str_view():
    time.sleep(random.uniform(2, 5)) # look human
    
    # Is there more than 1 result?
    if len(driver.find_elements_by_css_selector('.section-partial-interpretation-title')) > 0: # if there's multiple options
        print("Multiple results found\n")
        
        element_to_hover_over = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[2]/div[2]/div[2]/div[2]/div[1]')
        hover_click = ActionChains(driver).move_to_element(element_to_hover_over).send_keys(Keys.ENTER)
        
        #first_res = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[2]/div[2]/div[2]/div[2]/div[1]')
        #    //*[@id="pane"]/div/div[1]/div/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/h3/span
        #first_res.click() # click the first one
        #print(driver.find_elements_by_css_selector('.section-result-content'))
        print("Clicking first\n")
    
    # capture change (if made)
    before_str_view_url = str(driver.current_url)

    # Is there streetview available? 
    tmp_elem = driver.find_elements_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[1]/div[1]/button')
    if len(tmp_elem) > 0:
        tmp_elem[0].click() # try to click and see what happens
        
        if len(driver.find_elements_by_css_selector('#titlecard > div.widget-titlecard.widget-titlecard-show-settings-menu')) > 0:
        #str(driver.current_url) == before_str_view_url:
            print('Streetview exists\n')
            return(True)
        else:
            print('No Streetview\n')
            return(False)

Close side pane

In [41]:
def cls_side_pane():
    driver.find_element_by_xpath('//*[@id="pane"]/div/div[3]/button').click()
    time.sleep(random.uniform(1, 5)) # look human

Hide all besides the image background - **NOTICE** the mark stays 

In [42]:
def visib_hide():    
    # the water mark doesn't remove?
    css_elems = ['.widget-image-header-close',
                '.widget-image-header-scrim',
                '.watermark',
                '.app-viewcard-strip',
                '.scene-footer',
                '#pane',
                '#titlecard']
    
    for elem in css_elems:
        driver.execute_script(f"document.querySelector('{elem}').style.display = 'none';")
        
    time.sleep(random.uniform(1, 3)) # look human

In [43]:
def visib_show():    
    # the water mark doesn't remove?
    css_elems = ['.widget-image-header-close',
                '.widget-image-header-scrim',
                '.watermark',
                '.app-viewcard-strip',
                '.scene-footer',
                '#pane',
                '#titlecard']
    
    for elem in css_elems:
        driver.execute_script(f"document.querySelector('{elem}').style.display = 'unset';")
        
    time.sleep(random.uniform(1, 3)) # look human

In [44]:
def reset_for_search():
    visib_show()
    cls_side_pane() # here it actually 'opens' not as the name suggests
    time.sleep(random.uniform(1, 3)) # look human
    driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[1]/button[1]').click() # click to 'return'

Take screenshot

In [45]:
def scrn_shot(img_num, address):
    driver.save_screenshot(f"./final/str_view/{img_num}_{address}.png") # notice saved location

Pack functions into a larger function

In [46]:
def url_to_image(img_num, address):
    if clk_str_view(): # if ther's a streetview option
        print("Moving to Stretview\n")
        cls_side_pane()
        visib_hide()
        scrn_shot(img_num, address)
        reset_for_search() # go back to main search page > ready for the next search
    # either way, delete what you searched for
    search_bar = driver.find_element_by_xpath('//*[@id="searchbox-searchbutton"]') # find search bar
    search_bar.send_keys(Keys.CONTROL + "a", Keys.BACKSPACE) # first select and delete whatever text exists in the search bar 

# WIP

In [67]:
safe_address_lst = safe_mrg_df.address.tolist()
unsafe_address_lst = unsafe_df.address.tolist()

In [69]:
#unsafe_address_lst

In [80]:
unsafe_address_lst = unsafe_address_lst[10:] # just for tests

In [81]:
len(unsafe_address_lst)

278

In [83]:
driver = webdriver.Chrome(PATH)
driver.implicitly_wait(10)
driver.get(gmaps_url_base)
#open_gmaps(gmaps_url_base)
print('opened window')
accept_cookies()
print('accepted cookies')
for img_num, address in enumerate(unsafe_address_lst): # change according to what your 
    print(f'Scraping: {img_num} out of: {len(unsafe_address_lst)}. There are {len(unsafe_address_lst) - img_num} images left.\n')
    print(f'{address}\n')
    time.sleep(random.uniform(1, 5)) # look human
    #tmp_url = gmaps_url_base + address
    search_location(address)
    time.sleep(random.uniform(3, 10)) # look human
    url_to_image(img_num, address) # for ecach address
    time.sleep(random.uniform(3, 10)) # look human

opened window
CLICKED: cockies
accepted cookies
Scraping: 0 out of: 278. There are 278 images left.

5615 NETHERLAND AVENUE, BRONX, New York, New York

Streetview exists

Moving to Stretview

Scraping: 1 out of: 278. There are 277 images left.

75 LIVINGSTON STREET, BROOKLYN, New York, New York

Streetview exists

Moving to Stretview

Scraping: 2 out of: 278. There are 276 images left.

50-05 43 AVENUE, QUEENS, New York, New York

Streetview exists

Moving to Stretview

Scraping: 3 out of: 278. There are 275 images left.

215 EAST GUNHILL ROAD, BRONX, New York, New York

Streetview exists

Moving to Stretview

Scraping: 4 out of: 278. There are 274 images left.

14 WEST 57 STREET, MANHATTAN, New York, New York

Streetview exists

Moving to Stretview

Scraping: 5 out of: 278. There are 273 images left.

4200 AVENUE K, BROOKLYN, New York, New York

Streetview exists

Moving to Stretview

Scraping: 6 out of: 278. There are 272 images left.

360 WEST 43 STREET, MANHATTAN, New York, New Yor

In [82]:
driver.quit()

In [47]:
print(safe_mrg_df.address.tolist()[3])
geocode(safe_mrg_df.address.tolist()[3])


1293 YORK AVENUE, MANHATTAN, New York, New York


Location(1293, York Avenue, Lenox Hill, Manhattan Community Board 8, Manhattan, New York County, New York, 10021, United States, (40.7651843, -73.9551931, 0.0))

In [307]:
# NOT CURRENTLY WORKING 
try:
    driver = webdriver.Chrome(PATH)
    driver.implicitly_wait(10)
    driver.get(gmaps_url_base)
    #open_gmaps(gmaps_url_base)
    print('opened window')
    accept_cookies()
    print('accepted cookies')
    for img_num, address in enumerate(safe_address_lst):
        print(address, img_num)
        time.sleep(random.uniform(1, 5)) # look human
        #tmp_url = gmaps_url_base + address
        search_location(address)
        time.sleep(random.uniform(3, 10)) # look human
        url_to_image(img_num, address) # for ecach address
except:
    driver.quit()

opened window
CLICKED: cockies
accepted cookies
2937 8 AVENUE, MANHATTAN, New York, New York 0
